In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from kerastuner.tuners import RandomSearch

In [4]:
df = pd.read_csv("winequality-red.csv")

In [5]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [9]:
def model_selection(hp):
    model = keras.Sequential()
    for i in range(hp.Int("units_",2,10)):
        model.add(layers.Dense(
                        units = hp.Int("units_"+str(i),
                                      min_value = 32,
                                       max_value=512,
                                       step = 32
                                      ),
                        activation='relu'))
        model.add(layers.Dense(1,activation='linear'))
        model.compile(
                optimizer = keras.optimizers.Adam(
                        hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
                        loss = 'mean_absolute_error',
                        metrics = ['mean_absolute_error']
                    )
        return model
    
    
    

In [11]:
tuner = RandomSearch(
    model_selection,
    objective = 'val_mean_absolute_error',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'project',
    project_name = 'Red Wine Quality'
)

In [13]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
units_ (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 10, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, random_state = 0)

In [16]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_mean_absolute_error: 0.5177289446194967

Best val_mean_absolute_error So Far: 0.5177289446194967
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit


In [18]:
tuner.results_summary()

Results summary
Results in project/Red Wine Quality
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
units_: 9
units_0: 480
learning_rate: 0.01
Score: 0.5177289446194967
Trial summary
Hyperparameters:
units_: 4
units_0: 416
learning_rate: 0.01
Score: 0.5317826072374979
Trial summary
Hyperparameters:
units_: 10
units_0: 416
learning_rate: 0.0001
Score: 0.5518904725710551
Trial summary
Hyperparameters:
units_: 10
units_0: 320
learning_rate: 0.01
Score: 0.5668669740358988
Trial summary
Hyperparameters:
units_: 3
units_0: 64
learning_rate: 0.0001
Score: 3.3802829583485923
